# Ensemble Learning

## Initial Imports

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [17]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

In [18]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [19]:
df[df.columns[df.dtypes == "object"]]

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,RENT,Source Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
1,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
2,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
3,RENT,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
4,MORTGAGE,Not Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
...,...,...,...,...,...,...,...,...,...,...
68812,RENT,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68813,RENT,Not Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68815,MORTGAGE,Verified,Jan-2019,low_risk,n,f,May-2019,Individual,N,N


In [21]:
df.issue_d.value_counts(), df.next_pymnt_d.value_counts()

(Jan-2019    31041
 Feb-2019    25579
 Mar-2019    12197
 Name: issue_d, dtype: int64,
 May-2019    42449
 Apr-2019    26368
 Name: next_pymnt_d, dtype: int64)

In [22]:
obj_df = df[df.columns[df.dtypes == "object"]]
obj_df = pd.DataFrame({col: obj_df[col].astype('category').cat.codes for col in obj_df}, index=obj_df.index)
obj_df.head(10)

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,3,1,2,1,0,1,1,0,0,0
1,1,2,2,1,0,1,1,0,0,0
2,1,2,2,1,0,1,1,0,0,0
3,3,2,2,1,0,1,1,0,0,0
4,1,0,2,1,0,1,1,0,0,0
5,1,1,2,1,0,1,1,0,0,0
6,3,1,2,1,0,1,1,0,0,0
7,2,1,2,1,0,1,1,0,0,0
8,3,0,2,1,0,0,1,0,0,0
9,3,2,2,1,0,1,1,0,0,0


In [23]:
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


In [24]:
df.drop(list(df.columns[df.dtypes == "object"].values), axis = 1, inplace = True)
df = pd.concat([df, obj_df], axis = 1)
df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,3,1,2,1,0,1,1,0,0,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,2,2,1,0,1,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,2,2,1,0,1,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,3,2,2,1,0,1,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,2,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,3,1,1,1,0,1,1,0,0,0
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,3,0,1,1,0,1,1,0,0,0
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,1,1,1,1,0,1,1,0,0,0
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,1,2,1,1,0,0,1,0,0,0


In [25]:
# categorical_df.loan_status.value_counts()

## Split the Data into Training and Testing

In [26]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [27]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [28]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [29]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [30]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [31]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [70]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [71]:
# Calculated the balanced accuracy score
y_pred = rf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7493429827079996

In [72]:
balanced_accuracy_score(y_test, y_pred).round(4)

0.7493

In [73]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   65,    36],
       [ 2478, 14626]], dtype=int64)

In [74]:
# Print the imbalanced classification report 
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.64      0.86      0.05      0.74      0.54       101
          1       1.00      0.86      0.64      0.92      0.74      0.56     17104

avg / total       0.99      0.85      0.64      0.92      0.74      0.56     17205



In [75]:
# List the features sorted in descending order by feature importance
features_sorted = sorted(zip(rf.feature_importances_, X.columns), reverse=True)
features_sorted

[(0.07786803584622905, 'total_rec_prncp'),
 (0.07435679783168583, 'last_pymnt_amnt'),
 (0.057495323397011744, 'total_pymnt_inv'),
 (0.05390252952355928, 'total_rec_int'),
 (0.048535239541823746, 'total_pymnt'),
 (0.037282772204047125, 'int_rate'),
 (0.020892195001662523, 'issue_d'),
 (0.019304112323552492, 'mths_since_recent_inq'),
 (0.017902202114182673, 'dti'),
 (0.017289710176734373, 'out_prncp'),
 (0.016379915761619946, 'bc_util'),
 (0.01605716044419175, 'mths_since_rcnt_il'),
 (0.016033023274398196, 'max_bal_bc'),
 (0.015473815312242551, 'total_rev_hi_lim'),
 (0.015273954942763507, 'avg_cur_bal'),
 (0.015241704437788632, 'mo_sin_old_rev_tl_op'),
 (0.0148251130258712, 'total_bal_ex_mort'),
 (0.014704680615884494, 'installment'),
 (0.014496926610752631, 'out_prncp_inv'),
 (0.014479575029858846, 'total_bal_il'),
 (0.014340355338022909, 'tot_cur_bal'),
 (0.014307252254638143, 'il_util'),
 (0.01393416383658299, 'annual_inc'),
 (0.013755769764938856, 'mo_sin_old_il_acct'),
 (0.013464876

### Easy Ensemble Classifier

In [76]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
ee = EasyEnsembleClassifier(n_estimators=100, random_state=1, n_jobs=-1)
ee.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, n_jobs=-1, random_state=1)

In [80]:
# Calculated the balanced accuracy score
y_pred = ee.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9309000152821644

In [81]:
balanced_accuracy_score(y_test, y_pred).round(4)

0.9309

In [78]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [ 1009, 16095]], dtype=int64)

In [79]:
# Print the imbalanced classification report 
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.08      0.92      0.94      0.15      0.93      0.86       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.96      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier had the best accuracy score of 0.9309.

2. Which model had the best recall score?

    The Easy Ensemble Classifier had the best recall score of 0.94. 

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier had the best geo score of 0.93.

4. What are the top three features?

    The top three features are total_rec_prncp, last_pymnt_amnt and total_pymnt_inv.
    
    